In [42]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/raw/lending_club_loan_two.csv")

df.shape

(396030, 27)

In [43]:
df = df[df["loan_status"].isin(["Fully Paid", "Charged Off"])]

df["default_flag"] = df["loan_status"].apply(
    lambda x: 1 if x == "Charged Off" else 0
)

df["default_flag"].value_counts(normalize=True)

default_flag
0    0.803871
1    0.196129
Name: proportion, dtype: float64

In [46]:
drop_cols = [
    "grade", # internal scoring
    "sub_grade", # internal scoring
    "emp_title", # high cardinality
    "title", # redundant
    "address", # unused now
    "loan_status", # because we have default_flag
    "application_type" # because we only have one type
]

df = df.drop(columns=drop_cols)

df.shape

(396030, 21)

In [47]:
df["term"] = df["term"].str.extract(r"(\d+)").astype(int)

In [48]:
df["emp_length"] = df["emp_length"].str.replace("years", "", regex=False)
df["emp_length"] = df["emp_length"].str.replace("year", "", regex=False)
df["emp_length"] = df["emp_length"].str.replace("+", "", regex=False)
df["emp_length"] = df["emp_length"].str.replace("< 1", "0.5", regex=False)
df["emp_length"] = df["emp_length"].str.strip()
df["emp_length"] = pd.to_numeric(df["emp_length"], errors="coerce")

In [49]:
df["issue_d"] = pd.to_datetime(df["issue_d"], format="%b-%Y")
df["issue_year"] = df["issue_d"].dt.year

In [50]:
df["earliest_cr_line"] = pd.to_datetime(df["earliest_cr_line"], format="%b-%Y")

df["credit_history_years"] = (
    df["issue_d"].dt.year - df["earliest_cr_line"].dt.year
)

In [51]:
df = df.drop(columns=["issue_d", "earliest_cr_line"])

In [52]:
df["mort_acc"] = df["mort_acc"].fillna(df["mort_acc"].median())

In [53]:
df["emp_length"] = df["emp_length"].fillna(df["emp_length"].median())

In [54]:
if df["revol_util"].dtype == "object":
    df["revol_util"] = df["revol_util"].str.replace("%", "")
    df["revol_util"] = pd.to_numeric(df["revol_util"], errors="coerce")

df["revol_util"] = df["revol_util"].fillna(df["revol_util"].median())

In [55]:
df = pd.get_dummies(df, drop_first=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 38 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   loan_amnt                            396030 non-null  float64
 1   term                                 396030 non-null  int32  
 2   int_rate                             396030 non-null  float64
 3   installment                          396030 non-null  float64
 4   emp_length                           396030 non-null  float64
 5   annual_inc                           396030 non-null  float64
 6   dti                                  396030 non-null  float64
 7   open_acc                             396030 non-null  float64
 8   pub_rec                              396030 non-null  float64
 9   revol_bal                            396030 non-null  float64
 10  revol_util                           396030 non-null  float64
 11  total_acc    

In [57]:
df.isnull().sum().sum()

535

In [58]:
# The missing values is 535, and pub_rec_bankruptcies collum has 395495 values, 
# so the missing values is only 0.1% of the total values, 
# we can fill it with 0 because it means no public record of bankruptcies.
df["pub_rec_bankruptcies"] = df["pub_rec_bankruptcies"].fillna(0)

In [59]:
df.isnull().sum().sum()

0

In [74]:
print(df.dtypes.to_string())

loan_amnt                              float64
term                                     int32
int_rate                               float64
installment                            float64
emp_length                             float64
annual_inc                             float64
dti                                    float64
open_acc                               float64
pub_rec                                float64
revol_bal                              float64
revol_util                             float64
total_acc                              float64
mort_acc                               float64
pub_rec_bankruptcies                   float64
default_flag                             int64
issue_year                               int32
credit_history_years                     int32
home_ownership_MORTGAGE                   bool
home_ownership_NONE                       bool
home_ownership_OTHER                      bool
home_ownership_OWN                        bool
home_ownershi

In [60]:
df.to_csv("../data/processed/lending_club_clean.csv", index=False)